In [1]:
-- creating delta table
/*CREATE TABLE pmfd_db.fact_consolidated_financials 
    (
    consolidated_financials_sid   bigint NOT NULL,
    metric_date_sk                bigint NOT NULL,
    consolidation_group_unit_sk   bigint NOT NULL,
    budget_plan_version_sk        bigint NOT NULL,
    forecast_plan_version_sk      bigint NOT NULL,
    group_account_sk              bigint NOT NULL,
    actual_amount_usd             DECIMAL(38, 16),
    budget_amount_usd             DECIMAL(38, 16),
    forecast_amount_usd           DECIMAL(38, 16),
    is_latest_forecast_version    string) */


In [2]:
%sh git clone https://github.com/keshaan/myrepo.git 

Cloning into 'myrepo'...

In [3]:
%sh cd myrepo
 ls -al
 pwd

total 20
drwxr-xr-x 4 root root 4096 Jun 20 08:39 .
drwxr-xr-x 1 root root 4096 Jun 20 08:45 ..
drwxr-xr-x 8 root root 4096 Jun 20 08:39 .git
drwxr-xr-x 3 root root 4096 Jun 20 08:39 notebooks
-rw-r--r-- 1 root root 9 Jun 20 08:39 README.md
/databricks/driver/myrepo

In [4]:
-- Create temp table Finance_Consolidated_Financials_Full extract
create temporary view vw_Finance_Consolidated_Financials_ex as 
select right(FiscalPeriodYearCode_FSCPYRCD,4) Fiscal_year,int(substr(FiscalPeriodYearCode_FSCPYRCD,5,2)) Fiscal_Month,
GROUPACCOUNTCODE_GRPACCCD,CONSOLIDATIONGROUPCODE_CONGRPCD,CONSOLIDATIONUNITCODE_CONUNCD,PLANVERSIONCODEBUDGET_PLNVERCDBDT,
PLANVERSIONCODEMONTHLYFORECAST_PLNVERCDMONFC,ACTUAL_ACTUAL,BUDGET_BUDGET,MONTHLYCOMPARATIVEFORECAST_MTHCMPFC
from pmfd_db.Finance_Consolidated_Financials_Full

In [5]:
-- Temp table for dim date extract
create temporary view vw_dim_date as
select date_sk,AU_Fiscal_Year_Code,int(AU_Fiscal_Month_Code) from pmfd_db.dim_date
where Day_Number_Of_Month = 1

In [6]:
-- Temp table for adding date key attribute
create temporary view vw_date_Fin_con_fin as
select a.date_sk, b.* from vw_dim_date a, vw_Finance_Consolidated_Financials_ex b where a.AU_Fiscal_Year_Code=b.Fiscal_year and a.AU_Fiscal_Month_Code=b.Fiscal_Month

In [7]:
-- Temp table for Consolidate group extract
create temporary view vw_dim_con_grp_ex as
select Consolidation_Group_Unit_SK,Consolidation_Group_Code,Consolidation_Unit_Code from pmfd_db.DIM_Consolidation_Group

In [8]:
-- Tem table for adding consolidated group key attribute
create temporary view vw_date_con_fin_con_grp as
select a.date_sk, b.Consolidation_Group_Unit_SK, a.GROUPACCOUNTCODE_GRPACCCD,a.PLANVERSIONCODEBUDGET_PLNVERCDBDT, a.PLANVERSIONCODEMONTHLYFORECAST_PLNVERCDMONFC, a.ACTUAL_ACTUAL, a.BUDGET_BUDGET, a.MONTHLYCOMPARATIVEFORECAST_MTHCMPFC from vw_date_Fin_con_fin a, vw_dim_con_grp_ex b where a.CONSOLIDATIONGROUPCODE_CONGRPCD=b.Consolidation_Group_Code and a.CONSOLIDATIONUNITCODE_CONUNCD=b.Consolidation_Unit_Code

In [9]:
-- Temp table for budget extract
create temporary view vw_budget_version as
select plan_version_sk Budget_Plan_Version_SK, Plan_Type_Code Budget_Code from pmfd_db.dim_plan_type_version
where Plan_Version_Code = "BUDGET"

In [10]:
-- Temp table for forecast extract
create temporary view vw_forecast_version as
select plan_version_sk Forecast_Plan_Version_SK, Plan_Type_Code Forecast_Code from pmfd_db.dim_plan_type_version
where Plan_Version_Code = "FORECAST"

In [11]:
-- Temp table for adding budget key attribute
create temporary view vw_date_con_fin_con_grp_budg as
select a.date_sk, a.Consolidation_Group_Unit_SK,b.Budget_Plan_Version_SK,a.GROUPACCOUNTCODE_GRPACCCD,a.PLANVERSIONCODEMONTHLYFORECAST_PLNVERCDMONFC, a.ACTUAL_ACTUAL, a.BUDGET_BUDGET, a.MONTHLYCOMPARATIVEFORECAST_MTHCMPFC from vw_date_con_fin_con_grp a left outer join vw_budget_version b on a.PLANVERSIONCODEBUDGET_PLNVERCDBDT = b.Budget_Code

In [12]:
-- Temp table for adding forecast key attribute
create temporary view vw_date_con_fin_con_grp_budg_forc_plan as
select a.date_sk, a.Consolidation_Group_Unit_SK,a.Budget_Plan_Version_SK,b.Forecast_Plan_Version_SK, a.GROUPACCOUNTCODE_GRPACCCD, a.ACTUAL_ACTUAL, a.BUDGET_BUDGET, a.MONTHLYCOMPARATIVEFORECAST_MTHCMPFC from vw_date_con_fin_con_grp_budg a left outer join vw_forecast_version b on a.PLANVERSIONCODEMONTHLYFORECAST_PLNVERCDMONFC = b.Forecast_Code

In [13]:
-- Temp table for adding Group Account key attribute
create temporary view vw_date_con_fin_con_grp_budg_forc_plan_grp_acc as
select a.date_sk, a.Consolidation_Group_Unit_SK,a.Budget_Plan_Version_SK,a.Forecast_Plan_Version_SK, b.Group_Account_SK, a.ACTUAL_ACTUAL, a.BUDGET_BUDGET, a.MONTHLYCOMPARATIVEFORECAST_MTHCMPFC from vw_date_con_fin_con_grp_budg_forc_plan a left outer join  pmfd_db.dim_group_account b on a.GROUPACCOUNTCODE_GRPACCCD=b.group_account_code

In [14]:
-- Temp table to add Is_Latest_Forecast_Version attribute
create temporary view vw_date_con_fin_con_grp_budg_forc_plan_grp_acc_max_forc as
select date_sk, Consolidation_Group_Unit_SK,Group_Account_SK,max(Forecast_Plan_Version_SK) max_Forecast_Plan_Version_SK,"Y" Is_Latest_Forecast_Version from vw_date_con_fin_con_grp_budg_forc_plan_grp_acc group by date_sk,Consolidation_Group_Unit_SK,Group_Account_SK

In [15]:
-- Handling nulls in key columns
create temporary view vw_date_con_fin_con_grp_budg_forc_plan_grp_acc_max_forc_flag as
select a.date_sk, a.consolidation_group_unit_sk, if(a.budget_plan_version_sk is null,'-1',a.budget_plan_version_sk) budget_plan_version_sk,if(a.forecast_plan_version_sk is null, '-1',a.forecast_plan_version_sk) forecast_plan_version_sk, a.group_account_sk, a.actual_actual, a.budget_budget, a.MONTHLYCOMPARATIVEFORECAST_MTHCMPFC,if(b.Is_Latest_Forecast_Version is null,'N',Is_Latest_Forecast_Version) Is_Latest_Forecast_Version from vw_date_con_fin_con_grp_budg_forc_plan_grp_acc a left outer join vw_date_con_fin_con_grp_budg_forc_plan_grp_acc_max_forc b on a.date_sk=b.date_sk and a.Consolidation_Group_Unit_SK=b.Consolidation_Group_Unit_SK and a.Group_Account_SK=b.Group_Account_SK and a.Forecast_Plan_Version_SK=b.max_Forecast_Plan_Version_SK

In [16]:
-- Final Data insertion in fact consolidated financials
insert into table pmfd_db.fact_consolidated_financials
select row_number() over(order by date_sk) + (select max(consolidated_financials_sid) from pmfd_db.fact_consolidated_financials), a.* from vw_date_con_fin_con_grp_budg_forc_plan_grp_acc_max_forc_flag a